# Notebook rilut

### Bikin jelek, hapus aja

- semua yang categorical: colorCode, customerID, articleID, orderDate, productGroup

### Jangan dihapus

- price, rrp (walaupun impute median)

### Missing values

- rrp, productGroup

### Catatan:

Logistic Regression lama banget. itu pun solvernya udah diubah. kalau masih mau pake, iternya kurangin kali?

### TODO:

- ekstrak fitur karena fitur adalah segalanya

In [103]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import confusion_matrix
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from operator import itemgetter
#from sklearn.externals import joblib

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [24]:
# Pakai make_datasets_train_only. Buat sendiri juga boleh

df=pd.read_csv('train_gue2.csv')

if 'deviceID' in df.columns:
    del df['deviceID']
    
del df["colorCode"]
del df['customerID']
del df['articleID']
del df['orderDate']
#del df['productGroup']

    
if 'orderID' in df.columns:
    del df['orderID']

df.head()

,paymentMethod,price,productGroup,quantity,returnQuantity,rrp,sizeCode,voucherAmount,order_total,customer_budget,...,after_voucher,order_order,choice_order,articleID_cumprob,colorCode_cumprob,customerID_cumprob,sizeCode_cumprob,ac_cumprob,as_cumprob,cps_cumprob
0,BPRG,10.000000,3.0,1,0.0,29.990000,44,0.0,30.000000,28.500000,...,30.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BPRG,20.000000,3.0,1,0.0,39.990002,44,0.0,30.000000,28.500000,...,30.000000,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BPRG,35.000000,8.0,1,0.0,49.990002,38,0.0,84.989998,84.989998,...,84.989998,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BPRG,49.990002,8.0,1,1.0,49.990002,38,0.0,84.989998,84.989998,...,84.989998,2,2,1.0,1.0,0.5,0.5,1.0,1.0,0.5
4,PAYPALVC,10.000000,8.0,1,0.0,35.990002,42,0.0,60.000000,60.000000,...,60.000000,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# Drop kategorikal
object_columns=df.loc[:, df.dtypes == object].columns
df= df.drop(object_columns, axis=1)

# Turunin presisi, iseng-iseng aja
float_64_columns = df.loc[:, df.dtypes == np.float64].columns
for col in float_64_columns:
    df[col] = df[col].astype(np.float32)

int_64_columns = df.loc[:, df.dtypes == np.int64].columns
for col in int_64_columns:
    df[col] = df[col].astype(np.int32)
    

label='returnQuantity'
df.rrp.fillna(df.rrp.median(), inplace=True)
Xdf=df.drop(['productGroup','voucherAmount','after_voucher','order_order'], axis=1)
Xdf=Xdf.drop([label], axis=1)

X=Xdf.values
y=df.returnQuantity.values


X_train=X[:-341099]
X_test=X[-341099:]
y_train=y[:-341099]
y_test=y[-341099:]

In [100]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
#y_pred[y_pred>1.3]=y_pred[y_pred>1.3]+0.5
#y_pred[y_pred>2.4]=y_pred[y_pred>2.4]+0.3
#y_pred[y_pred>3.2]=y_pred[y_pred>3.2]+0.3
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)

[[143266  23644      9      0      0      0]
 [ 19466 154036      8      0      0      0]
 [     3    617     10      0      0      0]
 [     0     11     18      0      0      0]
 [     0      0      7      0      0      0]
 [     0      0      4      0      0      0]]
43825.0
jumlah data test:  341099


# 43825
Atas->Bawah: True, Kiri->Kanan: Predicted
```
[[143266  23644      9      0      0      0]
 [ 19466 154036      8      0      0      0]
 [     3    617     10      0      0      0]
 [     0     11     18      0      0      0]
 [     0      0      7      0      0      0]
 [     0      0      4      0      0      0]]
43825.0
jumlah data test:  341099
```

In [58]:
Xdf.columns.tolist()

['price',
 'quantity',
 'rrp',
 'order_total',
 'customer_budget',
 'expense_ratio',
 'unit_price',
 'median_unit_price',
 'price_diff',
 'choice_order',
 'articleID_cumprob',
 'colorCode_cumprob',
 'customerID_cumprob',
 'sizeCode_cumprob',
 'ac_cumprob',
 'as_cumprob',
 'cps_cumprob']

### Ini Importances?

In [50]:
from sklearn.feature_selection import RFE
lr=LinearRegression(n_jobs=-1, normalize=True)
selector = RFE(lr, 15, step=1)
selector = selector.fit(X, y)
pair=zip(Xdf.columns, selector.ranking_)
t1=[i for i in pair]; t1
sorted(t1, key=itemgetter(1))

[('price', 1),
 ('quantity', 1),
 ('rrp', 1),
 ('expense_ratio', 1),
 ('unit_price', 1),
 ('median_unit_price', 1),
 ('price_diff', 1),
 ('choice_order', 1),
 ('articleID_cumprob', 1),
 ('colorCode_cumprob', 1),
 ('customerID_cumprob', 1),
 ('sizeCode_cumprob', 1),
 ('ac_cumprob', 1),
 ('as_cumprob', 1),
 ('cps_cumprob', 1),
 ('customer_budget', 2),
 ('order_total', 3),
 ('order_order', 4)]

In [102]:
rfr=RandomForestRegressor(n_jobs=-1, n_estimators=20)
rfr.fit(X_train, y_train)
y_pred=rfr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)

[[142021  24897      1      0      0      0]
 [ 18092 155372     46      0      0      0]
 [    15    220    393      2      0      0]
 [     2      2     10     13      2      0]
 [     0      0      0      3      4      0]
 [     1      0      0      1      0      2]]
43321.0
jumlah data test:  341099


In [104]:
etr=ExtraTreesRegressor(n_jobs=-1, n_estimators=20)
etr.fit(X_train, y_train)
y_pred=etr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)

[[142154  24756      9      0      0      0]
 [ 18362 155092     54      2      0      0]
 [    14    215    399      2      0      0]
 [     1      4      9     14      1      0]
 [     0      0      1      1      5      0]
 [     1      0      1      0      0      2]]
43471.0
jumlah data test:  341099


In [113]:
lasso=Lasso(0.01)
lasso.fit(X_train, y_train)
y_pred=lasso.predict(X_test)
y_pred[y_pred>1.3]=y_pred[y_pred>1.3]+0.5
#y_pred[y_pred>2.4]=y_pred[y_pred>2.4]+0.3
#y_pred[y_pred>3.2]=y_pred[y_pred>3.2]+0.3
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)

[[142680  24230      9      0      0      0]
 [ 22354 151153      3      0      0      0]
 [    22    584     24      0      0      0]
 [     2     22      5      0      0      0]
 [     0      6      1      0      0      0]
 [     0      2      1      1      0      0]]
47321.0
jumlah data test:  341099


In [114]:
y_pred=lasso.predict(X_test)
y_pred[y_pred>1.3]=y_pred[y_pred>1.3]+1
#y_pred[y_pred>2.4]=y_pred[y_pred>2.4]+0.3
#y_pred[y_pred>3.2]=y_pred[y_pred>3.2]+0.3
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)

[[142680  24225      5      9      0      0]
 [ 22354 151151      2      3      0      0]
 [    22    536     48     24      0      0]
 [     2     20      2      5      0      0]
 [     0      4      2      1      0      0]
 [     0      2      0      1      1      0]]
47304.0
jumlah data test:  341099


In [ ]:
# Berat banget
"""from sklearn.linear_model import TheilSenRegressor
tsr=TheilSenRegressor( n_jobs=3)
tsr.fit(X_train, y_train)
y_pred=tsr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
print 'jumlah data test: ', len(y_test)"""

```

```